In [1]:
import pandas as pd
import sqlite3

# load your cleaned data
df = pd.read_csv('C:/Users/maana/OneDrive/Desktop/malaysia-job-market-analysis/data/cleaned/jobstreet_cleaned.csv')

# create a local database
conn = sqlite3.connect('jobstreet.db')

# load the data into SQL table
df.to_sql('jobs', conn, if_exists='replace', index=False)

print("Database created successfully!")

# test it works
result = pd.read_sql("SELECT COUNT(*) as total_jobs FROM jobs", conn)
print(result)

Database created successfully!
   total_jobs
0       69024


In [2]:
query1 = """
WITH category_stats AS (
    SELECT 
        category,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary,
        ROUND(MIN(salary_min), 0) as min_salary,
        ROUND(MAX(salary_max), 0) as max_salary,
        SUM(CASE WHEN salary_avg IS NOT NULL THEN 1 ELSE 0 END) as jobs_with_salary
    FROM jobs
    GROUP BY category
),
ranked AS (
    SELECT *,
        RANK() OVER (ORDER BY avg_salary DESC) as salary_rank,
        RANK() OVER (ORDER BY total_jobs DESC) as demand_rank
    FROM category_stats
    WHERE jobs_with_salary >= 50
)
SELECT 
    category,
    total_jobs,
    demand_rank,
    avg_salary,
    salary_rank,
    min_salary,
    max_salary
FROM ranked
ORDER BY avg_salary DESC
"""

result1 = pd.read_sql(query1, conn)
print(result1.to_string(index=False))

                              category  total_jobs  demand_rank  avg_salary  salary_rank  min_salary  max_salary
              CEO & General Management         120           25     22741.0            1      2000.0    500000.0
                Real Estate & Property         705           17      6992.0            2       800.0    100000.0
Information & Communication Technology        8675            3      6422.0            3         0.0    180000.0
                 Consulting & Strategy         315           22      5844.0            4       700.0     20000.0
          Banking & Financial Services        2641            9      5784.0            5         0.0    150000.0
                                 Sales        5715            5      5708.0            6         0.0    300000.0
                  Science & Technology         551           19      5617.0            7       500.0    160000.0
                          Construction        1621           11      5425.0            8       7

In [3]:
# Query 2 - salary by location with job count
query2 = """
WITH location_stats AS (
    SELECT
        location,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary,
        SUM(CASE WHEN job_type = 'Full time' THEN 1 ELSE 0 END) as fulltime_jobs
    FROM jobs
    GROUP BY location
    HAVING total_jobs >= 200
)
SELECT *,
    RANK() OVER (ORDER BY avg_salary DESC) as salary_rank
FROM location_stats
ORDER BY avg_salary DESC
LIMIT 15
"""

result2 = pd.read_sql(query2, conn)
print(result2.to_string(index=False))

                              location  total_jobs  avg_salary  fulltime_jobs  salary_rank
                                Penang        1930      7216.0           1832            1
              Kuala Lumpur City Centre        2112      6217.0           2032            2
                          Kuala Lumpur       15390      5468.0          13646            3
                         Bangsar South         514      5268.0            489            4
               Petaling Jaya, Selangor         519      5253.0            474            5
Kuala Lumpur City Centre, Kuala Lumpur         438      5217.0            410            6
                                Sepang         865      5149.0            702            7
                               Bangsar         254      5091.0            225            8
                             Putrajaya         242      5053.0            195            9
                      Kuantan District         305      5020.0            222           10

In [4]:
# Query 3 - ICT jobs specifically, salary by role
query3 = """
WITH ict_roles AS (
    SELECT
        subcategory,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary
    FROM jobs
    WHERE category = 'Information & Communication Technology'
    AND salary_avg IS NOT NULL
    GROUP BY subcategory
    HAVING total_jobs >= 20
)
SELECT *,
    RANK() OVER (ORDER BY avg_salary DESC) as salary_rank
FROM ict_roles
ORDER BY avg_salary DESC
"""

result3 = pd.read_sql(query3, conn)
print(result3.to_string(index=False))

                          subcategory  total_jobs  avg_salary  salary_rank
                Engineering - Network          20     13358.0            1
       Programme & Project Management         124      8391.0            2
                   Sales - Pre & Post          57      7746.0            3
               Engineering - Software         390      7492.0            4
                             Security          78      7423.0            5
                          Consultants          60      7192.0            6
               Developers/Programmers         632      6747.0            7
                           Management         147      6420.0            8
Database Development & Administration          55      6334.0            9
          Testing & Quality Assurance         102      6323.0           10
     Product Management & Development          55      6166.0           11
            Business/Systems Analysts         182      6108.0           12
                         